In [1]:
#libraries
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from matplotlib import pyplot
eps = np.finfo(float).eps
from numpy import log2 as log
# Quick value count calculator
from collections import Counter

In [2]:

#a function for read data set
df = pd.read_csv("/Users/najmeh/Desktop/master/Ml/projet/glass.csv")
features = df.columns[:-1].tolist()
print(df.shape)
df.head(2)

#discrete data
import pandas as pd
def discretization(self):
        exc = pd.ExcelFile(self.file_location).parse(self.sheet)
        exc['RI']=pd.cut(exc['RI'],3,labels = ['low', 'Normal','High'])
        exc['Na']=pd.cut(exc['Na'],3,labels = ['low', 'Normal','High'])
        exc['Mg']=pd.cut(exc['Mg'],3,labels = ['low', 'Normal','High'])
        exc['Al']=pd.cut(exc['Al'],3,labels = ['low', 'Normal','High'])
        exc['Si']=pd.cut(exc['Si'],3,labels = ['low', 'Normal','High'])
        exc['K']=pd.cut(exc['K'],3,labels = ['low', 'Normal','High'])
        exc['Ca']=pd.cut(exc['Ca'],3,labels = ['low', 'Normal','High'])
        exc['Ba']=pd.cut(exc['Ba'],3,labels = ['low', 'Normal','High'])
        exc['Fe']=pd.cut(exc['Fe'],3,labels = ['low', 'Normal','High'])
        exc['Type']=pd.cut(exc['Type'],7,labels = ['building_windows_float_processed', 'building_windows_non_float_processed','vehicle_windows_float_processed','vehicle_windows_non_float_processed ', 'containers','tableware','headlamps'])
        exc.to_excel(self.save_file_location,index=False)

#transform data six class 1,2,3,5,6,7 in target
pd.cut(df['Type'], bins=6).value_counts()
df['Type'].plot(kind='hist')

#spilit data to two group(training and test)

training_data = df.sample(frac=0.7, random_state=25)
testing_data = df.drop(training_data.index)

print(f"number of training examples: {training_data.shape[0]}")
print(f"number of testing examples: {testing_data.shape[0]}")

In [ ]:
#calculate entropy
entropy_node = 0  
#Unique objects - 1,2,3,5,6,,7
#here attribute = 'Type'
values = df.Type.unique()  
for value in values:
    fraction = df.Type.value_counts()[value]/len(df.Type)  
    entropy_node += -fraction*np.log2(fraction)

In [ ]:
#show us = entropy of the whole dataset
entropy_node

In [ ]:
def ent(df,attribute):
    target_variables = df.Type.unique()  #This gives all 'Yes'=1 and 'No'=0
    variables = df[attribute].unique()    


    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df.Type ==target_variable]) #numerator
            den = len(df[attribute][df[attribute]==variable])  
            fraction = num/(den+eps)  #pi
            entropy_each_feature += -fraction*log(fraction+eps) 
        fraction2 = den/len(df)
        entropy_attribute += -fraction2*entropy_each_feature   #Sums up all the entropy ETaste

    return(abs(entropy_attribute))

In [ ]:
#find entropy of all attribute
a_entropy = {k:ent(df,k) for k in df.keys()[:-1]}
a_entropy

In [ ]:
#define a function for calculate information gain
def ig(e_dataset,e_attr):
    return(e_dataset-e_attr)
    

In [ ]:
IG = {k:ig(entropy_node,a_entropy[k]) for k in a_entropy}
t22=[]

In [ ]:
def find_entropy(df):
    Class = df.keys()[-1]  
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  
  
def find_entropy_attribute(df,attribute):
#To make the code generic, changing target variable class name
  Class = df.keys()[-1]  
  #This gives all 'Yes' and 'No'
  target_variables = df[Class].unique()  
  #This gives different features in that attribute (like '0','y' in sex)
  variables = df[attribute].unique()    
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)

#find an attribute that has max ig
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        #Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]
  
#get subtable for a node
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)

# a function for build DT
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   
    
    

    #attribute maximum information gain
    node = find_winner(df)
    
   
    attValue = np.unique(df[node])
    
    #create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Type'],return_counts=True)      

        #Checking purity of subset that purity shoould be 0
        if len(counts)==1:
            tree[node][value] = clValue[0]
            
        else:        
            #Calling the function recursively 
            tree[node][value] = buildTree(subtable) 
                   
    return tree
  
  

## D-Tree 

In [ ]:
t=buildTree(df)
t

In [ ]:
def pred(inst,tree):
    for nodes in tree.keys():
        
        value=inst[nodes]
        
        tt=tree[nodes][value]
        
        prediction=0
        
        if type(tt) is dict:
            prediction = pred(inst,tt)
        else:
            prediction = tt
            break;
            
    return prediction
    

import pprint
q=df.shape[0]
acc=0
pp=[]

for i in range(q):
    ss=df.loc[i, : ]
    prediction = pred(ss,t)
    if prediction != ['Type']:
        acc=acc+1
        pp=np.append(pp,i)
        
pprint.pprint(t)
print('Accuracy = ')
print(1-(acc/df.shape[0]))
print('missclassified data= ')
pprint.pprint(pp)